In [ ]:
import pickle
import requests
from kiteconnect import KiteConnect
import logging
import datetime
import numpy as np
import pandas as pd
import ftplib
from datetime import date
today = date.today()
print(str(today))

In [ ]:
#### Connect to Data Feed #######
##################
kite = KiteConnect(api_key = "key") 
logging.basicConfig(level=logging.DEBUG)
kite.login_url()

In [ ]:
data = kite.generate_session("key", api_secret="key")
kite.set_access_token(data["access_token"])
token = data["access_token"]


In [ ]:
instrument_dump = kite.instruments("NSE")
def instrumentLookup(symbol):
    instrument_df = pd.DataFrame(instrument_dump)
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1



In [ ]:
a_file = open('/home/ec2-user/trading/Trade/Positions/postion_dict2021-07-16.pkl','rb')
position_dict = pickle.load(a_file)

##### The Entry Portfolio Value #####
trade_token_list = [] 
trade_symbolist = []
for key in position_dict.keys():
    trade_symbolist.append(key.upper())
    trade_token_list.append(instrumentLookup(key.upper()))
   

In [ ]:
##########Only if error in previous cell #############
######################## if pickle dict not created then create dict from trade file ###########################
df = pd.read_csv('/home/ec2-user/trading/Trade/Positions/total_orders2021-07-05.csv', names =  ['index','symbol','qty','client','type','date','time-stamp'])
df1 = df[df.client == '14FD050']
position_dict = {}
for sym,qty in zip(df1.symbol, df1.qty):
    position_dict.update({sym.lower().replace('-eq',''):qty})
trade_token_list = [] 
trade_symbolist = []
for key in position_dict.keys():
    trade_symbolist.append(key.upper())
    trade_token_list.append(instrumentLookup(key.upper()))

In [ ]:
b_file = open('/home/ec2-user/trading/Trade/Positions/position_dict'+ str(today) + '.pkl','wb')
pickle.dump(position_dict, b_file)
position_dict


In [ ]:
latest = []#iniatiing list
ltpdict=kite.ltp(trade_token_list)
for token in trade_token_list:  
    latest.append(ltpdict[str(token)]['last_price'])

In [ ]:
current_portfolio = 0 
current_equity_dict = {}
for key,lat in zip(position_dict.keys(),latest):
    current_portfolio += position_dict[key]*lat
    current_equity_dict.update({key.upper() + "-EQ":lat})

In [ ]:
##### Live Server
def stream_order():
    


    # create a new FTP() instance
    session = ftplib.FTP()

    # connect to our FTP site
    session.connect('14.142.196.162' )

    # log into the FTP site
    session.login('administrator', 'Hello@123!')
    
    ####### FINAL #####
    file = open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv','rb') 
    session.storbinary('STOR orders_stream.csv', file)     # send the file
    file.close()
    
    
    file = "/home/ec2-user/trading/Trade/Positions/orders_stream.csv"
    fil = open(file,"r")
    orders = fil.read()
    fil.close()
    
    wrt = open('/home/ec2-user/trading/Trade/Positions/total_orders' + str(today) + '.csv', 'a') 
    wrt.write(orders)
    wrt.close()
                
    #file = open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv','w+')
    #file.close()
    print("streamed :D")


In [ ]:
def stream_mid_office():

    # create a new FTP() instance
    session = ftplib.FTP()

    # connect to our FTP site
    session.connect('3.108.109.191')

    # log into the FTP site
    session.login('analyst', 'Dplr23#!')


    file = open('/home/ec2-user/trading/Trade/Positions/current.csv','rb') 
    session.storbinary('STOR /home/analyst/Current/current.csv', file)     # send the file
    file.close()
    
    for client_id in id_list:
        for file in file_type:
            #print(path_format(client_id, file)[1])
            localfile = open(path_format(client_id,file)[1], 'wb')
            #print(path_format(client_id, file)[0])
            session.retrbinary('RETR ' + path_format(client_id, file)[0], localfile.write, 1024)

    session.quit()
    localfile.close()



    
    
    

In [ ]:
def get_multiplier(idx):
    account_dict = {"14FD050":1, "14FD051" : 1, "14FD052": 1, "14FD053": 1, "14FD055": 1}
    #print(account_dict[idx])
    return account_dict[idx]

In [ ]:
import csv
import random
position_dict = {}
id_list = ["14FD050", "14FD051", "14FD052", "14FD053", "14FD055"]
#id_list = ["14FD050"]



def write_order_multi(symbol,perc,price,pos):    
    
    capital = 500000
    posnew=int(capital*perc/price) 
    #print(posnew)
    global order_index
    global current_time
    now = datetime.now(tz=ist)
    current_time = now.strftime("%H:%M:%S")
    position_dict.update({symbol:posnew})
    with open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv', 'a', newline='') as file: 
        writer = csv.writer(file)
        ################

        quantity=posnew-pos
        qty=int(quantity)
        
        random.shuffle(id_list)

        for idx in id_list:
            multiplier = get_multiplier(idx)
        
        
            if posnew*pos>=0:
                if qty==0:
                    pass
                elif qty>0:

                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

                else:
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

            elif posnew*pos<0:
                if pos>0:
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

                else: 
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

In [ ]:
#### on Monday enter Manually
order_index = 1

In [ ]:
######## commands to make FTP requests #######################
file_type = ['_NET_','_MTM_','_flaggedStocks_', '_exitFlag_',]
id_list = ['14FD050', '14FD051', '14FD052', '14FD053','14FD055']
def path_format(client_id, file_type):
    read_path = '/home/analyst/mid_office/' + client_id + '/' + client_id + file_type + str(today) + '.csv'
    write_path = '/home/ec2-user/trading/Trade/Positions/Equity/' + client_id + '/' + client_id + file_type + str(today) + '.csv'
    
    return read_path,write_path

In [ ]:
a_file =  open('/home/ec2-user/trading/Trade/Positions/position_dict' + str(today) + '.pkl','rb')
position_dict = pickle.load(a_file)
##### The Entry Portfolio Value #######
entry_portfolio = 0 
for key,lat in zip(position_dict.keys(),latest):
    entry_portfolio += position_dict[key]*lat
##### The Current Portfolio Value ######

import time

while True:
    print("Entering Loop")
    latest = [] #iniatiing list
    ltpdict=kite.ltp(trade_token_list)
    for token in trade_token_list:  
        latest.append(ltpdict[str(token)]['last_price']) 
    current_portfolio = 0 
    for key,lat in zip(position_dict.keys(),latest):
        current_portfolio += position_dict[key]*lat
        
    print(current_portfolio)
    
    if current_portfolio <= 0.97*entry_portfolio:
       ####### Clear Existing Positions ##########
        existing_position = []
        trade_symbolist = []
        for key in position_dict.keys():
            existing_position.append(position_dict[key])
            trade_symbolist.append(key.replace("-EQ",""))
        weight_list = np.zeros(len(existing_position))

        latest = [] #iniatiing list
        trade_token_list = [] 

        for sym in trade_symbolist:
            trade_token_list.append(instrumentLookup(sym.upper()))  
        #print(trade_token_list)
        ltpdict=kite.ltp(trade_token_list)    
        for token in trade_token_list:
            #print(token)
            latest.append(ltpdict[str(token)]['last_price']) 

        for symbol,perc,price,pos in zip(trade_symbolist,weight_list,latest,existing_position):
            write_order_multi(symbol,perc,price,pos)
        ####### Stream Order #######
        stream_order()

        break
    
    df = pd.DataFrame(columns = ['Symbol', 'Price'])

    symbolist = []
    for i in position_dict.keys():
        symbolist.append(i.upper())

    for i,j in zip(range(len(symbolist)), range(len(latest))):
        df.loc[i] = [symbolist[i],latest[j]]

    df.to_csv('/home/ec2-user/trading/Trade/Positions/current.csv', header = False)
    path = '/home/ec2-user/trading/Trade/Positions/current.csv'
   
    with open(path) as f:
        lines = f.readlines()
        last = len(lines) - 1
        lines[last] = lines[last].replace('\r','').replace('\n','')
    with open(path, 'w') as wr:
        wr.writelines(lines)
    #stream_mid_office()
    
        
        
    
    time.sleep(1)
       